# Sprint 3 – Webinar 7: KPIs financieros con SQL (90 min)
**Propósito:** usar SQL para calcular métricas financieras (ingresos, costos, margen, ROI), estructurar y comunicar resultados, y analizar tendencias.

**DB de práctica:** esquema de librería (books, authors, publishers, ratings, reviews). Para métricas financieras crearemos *CTEs* con datos de ejemplo mediante `VALUES` para no depender de tablas adicionales. Reemplaza esos CTE por tus tablas reales cuando las tengas.

**Conexión:** SQLAlchemy + `pd.read_sql()`.

**Duración sugerida:** 90 min (40 teoría + 35 práctica guiada + 15 ejercicios de entrega).



## Agenda
1) Conexión a la base y vista del esquema

2) **Calculating Key Financial Metrics**

3) **Structuring, Delivering and communicating Financial Reports**

4) **Analyzing Trends**

5) Cierre + tareas


## Esquema de práctica
Usaremos las tablas del diagrama (libros, autores, editoriales, ratings, reseñas).

<div style="text-align: center">
    <img src="https://raw.githubusercontent.com/ljpiere/tpdata_python/main/images/sp3_w2p.jpg" width="400">
</div>


In [ ]:
from sqlalchemy import create_engine
import pandas as pd

pd.options.display.max_columns = None

db_config = {
 'user': 'practicum_student', # username
 'pwd': 'QnmDH8Sc2TQLvy2G3Vvh7', # password
 'host': 'yp-trainers-practicum.cluster-czs0gxyx2d8w.us-east-1.rds.amazonaws.com',
 'port': 5432, 
 'db': 'data-analyst-final-project-db'
}

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db']
)

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [ ]:
# Helper: ejecuta una consulta y devuelve un DataFrame
def sql(query: str):
    return pd.read_sql(query, con=engine)

# Prueba rápida: contar libros por editorial
sql("""
SELECT p.publisher, COUNT(*) AS n_books
FROM books b
JOIN publishers p USING(publisher_id)
GROUP BY 1
ORDER BY 2 DESC
LIMIT 5;
""")

# 3. Calculating Key Financial Metrics

### 3.1 Aggregating Revenue and Cost Data
- Sumar ventas y costos por categorías de negocio.
- Filtrar por tiempo (ej.: último trimestre).
- Comparar revenue vs cost con alias/subconsultas.

**Nota:** Creamos un CTE `sales` con datos *mock* (book_id, sale_date, revenue, cost, campaign_id). Sustituye por tus tablas reales si existen.


In [ ]:
query = '''
WITH sales(book_id, sale_date, revenue, cost, campaign_id) AS (
    VALUES 
    (1, '2025-04-05', 120.00, 70.00, 101),
    (2, '2025-05-12', 250.00, 120.00, 102),
    (3, '2025-06-01', 175.00, 90.00, 101),
    (1, '2025-06-20', 90.00,  50.00, 103),
    (2, '2025-06-25', 140.00, 80.00, 103)
),
last_q AS (
  SELECT *
  FROM sales
  WHERE sale_date >= (date_trunc('quarter', CURRENT_DATE) - interval '1 quarter')
    AND sale_date < date_trunc('quarter', CURRENT_DATE)
)
SELECT p.publisher AS category,
       ROUND(SUM(lq.revenue),2) AS total_revenue,
       ROUND(SUM(lq.cost),2)    AS total_cost
FROM last_q lq
LEFT JOIN books b USING(book_id)
LEFT JOIN publishers p USING(publisher_id)
GROUP BY 1
ORDER BY total_revenue DESC;
'''
sql(query)

#### Exercise 3.1
Escribe una consulta que muestre **total revenue** y **total cost por *title*** para el trimestre pasado. (Tip: reutiliza el CTE `sales` y cambia el `GROUP BY`).

> Escribe tu solución aquí:

In [ ]:
# tu código
# sql(""" ... """)

### 3.2 Calculating Profit and Margin
- Crear campos con operaciones aritméticas.
- `gross_profit = revenue - cost`.
- `margin_pct = (revenue - cost) / NULLIF(revenue,0)`.


In [ ]:
query = '''
WITH sales(book_id, sale_date, revenue, cost, campaign_id) AS (
    VALUES 
    (1, '2025-04-05', 120.00, 70.00, 101),
    (2, '2025-05-12', 250.00, 120.00, 102),
    (3, '2025-06-01', 175.00, 90.00, 101),
    (1, '2025-06-20', 90.00,  50.00, 103),
    (2, '2025-06-25', 140.00, 80.00, 103)
)
SELECT p.publisher AS category,
       ROUND(SUM(revenue),2) AS revenue,
       ROUND(SUM(cost),2)    AS cost,
       ROUND(SUM(revenue - cost),2) AS gross_profit,
       ROUND(100 * SUM(revenue - cost)/NULLIF(SUM(revenue),0),2) AS margin_pct
FROM sales s
LEFT JOIN books b USING(book_id)
LEFT JOIN publishers p USING(publisher_id)
GROUP BY 1
ORDER BY margin_pct DESC;
'''
sql(query)

#### Exercise 3.2
Extiende tu consulta anterior para incluir **gross profit** y **margin percentage por categoría**.

In [ ]:
# tu código
# sql(""" ... """)

### 3.3 Measuring ROI by Campaign
- Unir ingresos y gasto de marketing por `campaign_id`.
- `ROI = (revenue - spend) / NULLIF(spend,0)`.
- Rankear campañas de mayor a menor.


In [ ]:
query = '''
WITH sales(book_id, sale_date, revenue, cost, campaign_id) AS (
    VALUES 
    (1, '2025-06-01', 200.00, 110.00, 501),
    (2, '2025-06-03', 180.00, 95.00, 502),
    (1, '2025-06-10', 160.00, 85.00, 501),
    (3, '2025-06-14', 90.00,  55.00, 503)
),
marketing_spend(campaign_id, spend) AS (
    VALUES (501, 150.00), (502, 220.00), (503, 60.00)
)
SELECT s.campaign_id,
       ROUND(SUM(s.revenue),2) AS revenue,
       ROUND(SUM(s.cost),2)    AS cost,
       ms.spend,
       ROUND(100 * (SUM(s.revenue) - ms.spend)/NULLIF(ms.spend,0),2) AS roi_pct
FROM sales s
JOIN marketing_spend ms USING(campaign_id)
GROUP BY s.campaign_id, ms.spend
ORDER BY roi_pct DESC;
'''
sql(query)

#### Exercise 3.3
Calcula el ROI de cada campaña y **ordénalas de mayor a menor**. (Puedes partir de la consulta anterior y ajustar números).

In [ ]:
# tu código
# sql(""" ... """)

### 3.4 Validating and Verifying SQL Results
- Compara totales vs. datos crudos.
- Verifica cálculos y filtros.
- Haz *sanity checks* siempre.

**Checklist rápido**
1) ¿Los totales de la agregación coinciden con la suma de la tabla base?
2) ¿Hay `NULL`s inesperados? Usa `COALESCE`.
3) ¿Las fechas y períodos están correctamente filtrados?
4) ¿Las métricas son invariantes ante `JOIN` duplicadores? Usa *distinct* o *grain* correcto.


In [ ]:
# Ejemplo de cross-check entre agregado y detalle
query_total = '''
WITH sales(book_id, sale_date, revenue, cost, campaign_id) AS (
    VALUES (1,'2025-06-01',200,110,501),(2,'2025-06-03',180,95,502),(1,'2025-06-10',160,85,501)
)
SELECT ROUND(SUM(revenue),2) AS total_revenue
FROM sales;
'''
total_df = sql(query_total)

query_by_cat = '''
WITH sales(book_id, sale_date, revenue, cost, campaign_id) AS (
    VALUES (1,'2025-06-01',200,110,501),(2,'2025-06-03',180,95,502),(1,'2025-06-10',160,85,501)
)
SELECT p.publisher, ROUND(SUM(s.revenue),2) AS revenue
FROM sales s
JOIN books b USING(book_id)
JOIN publishers p USING(publisher_id)
GROUP BY 1;
'''
by_cat_df = sql(query_by_cat)

total_df, by_cat_df, by_cat_df['revenue'].sum()

# 4. Structuring, Delivering and comunicating Financial Reports

### 4.1 Organizing SQL Output for Reporting
- Aliases descriptivos.
- Orden consistente.
- Listo para exportar (`df.to_csv`).


In [ ]:
df = sql("""
WITH sales(book_id, sale_date, revenue, cost, campaign_id) AS (
  VALUES (1,'2025-06-01',200,110,501),(2,'2025-06-03',180,95,502),(1,'2025-06-10',160,85,501)
)
SELECT p.publisher AS category,
       DATE_TRUNC('month', sale_date)::date AS month,
       ROUND(SUM(revenue),2) AS revenue_usd,
       ROUND(100 * SUM(revenue - cost)/NULLIF(SUM(revenue),0),2) AS margin_pct
FROM sales s
JOIN books b USING(book_id)
JOIN publishers p USING(publisher_id)
GROUP BY 1,2
ORDER BY month, category;
""")
df.head()

In [ ]:
# Exportar a CSV listo para slide o dashboard
# df.to_csv('financial_report.csv', index=False)
# print('Archivo guardado:', 'financial_report.csv')

#### Exercise 4.1
Ajusta una consulta para incluir **aliases significativos** y **orden** para exportar.

In [ ]:
# tu código

### 4.2 Documenting SQL Queries with Comments
- Usa comentarios `-- ...` y secciones.
- Documenta supuestos y filtros.


In [ ]:
sql("""
-- Métrica: margen por editorial en el mes actual
WITH sales(book_id, sale_date, revenue, cost, campaign_id) AS (
  -- Datos de ejemplo; reemplaza por tu tabla de ventas real
  VALUES (1,'2025-06-01',200,110,501),(2,'2025-06-03',180,95,502),(1,'2025-06-10',160,85,501)
)
SELECT 
  p.publisher                 AS category,      -- agrupación de negocio
  ROUND(SUM(revenue - cost),2) AS gross_profit, -- ingreso - costo
  ROUND(100 * SUM(revenue - cost)/NULLIF(SUM(revenue),0),2) AS margin_pct -- margen%
FROM sales s
JOIN books b USING(book_id)   -- une para conocer la editorial
JOIN publishers p USING(publisher_id)
WHERE sale_date >= date_trunc('month', CURRENT_DATE) -- filtro temporal
GROUP BY 1
ORDER BY margin_pct DESC;
""")

### 4.3 Summarizing and Presenting Business Recommendations – Part I
- Selecciona *lo más relevante*.
- Explica en lenguaje de negocio.
- Un gráfico + un párrafo.


In [ ]:
# Template de memo (completa con tus números)
from IPython.display import Markdown

Markdown("""
**Resumen ejecutivo (1 párrafo):**  
En el último mes, la editorial **X** presenta el mayor margen (**Y%**), impulsada por **título A**. Recomendamos **duplicar presupuesto** en campañas **501–503** dado su ROI > **Z%**, y revisar costos de distribución en **editorial W**.

**Gráfico sugerido:** tabla o barra de margen por editorial (exporta desde tu DataFrame).
""")

### 4.4 Summarizing and Presenting Business Recommendations – Part II
- Propón acciones basadas en margen/ROI/tendencias.
- Acompaña con tablas o visuales.


### 4.5 Adapting Messages for Different Stakeholders
Buenas prácticas:
- Claridad y concisión; títulos informativos.
- Ajusta nivel técnico al público.
- Para C‑level: **conclusiones primero**.

**Exercise 4.5**: Reescribe una recomendación en dos versiones.
- **Finance executive (CFO):** foco en margen, ROI, riesgo.
- **Marketing manager:** foco en campañas, audiencias, próximos pasos.

Completa aquí:

- **CFO:** _tu párrafo_
- **Marketing:** _tu párrafo_


### 4.6 AI 🤖 – Using LLMs to create summary slides
- Usa un LLM (ChatGPT) para bosquejar slides.
- Herramientas: Gamma.app para ejecutivos rápidos.
- Entrega resultados con formato profesional.


# 5. Analyzing Trends

### 5.1 Tracking KPIs Over Time
- `GROUP BY` por mes/trimestre.
- Comparar revenue y margin.
- Rolling averages, WoW/MoM/YoY.


In [ ]:
sql("""
WITH sales(book_id, sale_date, revenue, cost, campaign_id) AS (
  VALUES (1,'2025-01-10',120,70,501),(1,'2025-02-15',200,110,501),
         (2,'2025-03-01',180,95,502),(2,'2025-04-05',220,120,502),
         (3,'2025-05-20',90,55,503),(1,'2025-06-02',160,85,501)
)
SELECT date_trunc('month', sale_date)::date AS month,
       ROUND(SUM(revenue),2) AS revenue,
       ROUND(100*SUM(revenue-cost)/NULLIF(SUM(revenue),0),2) AS margin_pct,
       ROUND(AVG(SUM(revenue)) OVER(ORDER BY date_trunc('month', sale_date)
            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW),2) AS rolling_3m_revenue
FROM sales
GROUP BY 1
ORDER BY 1;
""")

#### Exercise 5.1
Muestra **revenue** y **margin% por mes** para los últimos 6 meses (usa `WHERE sale_date >= CURRENT_DATE - interval '6 months'`).

In [ ]:
# tu código

### 5.2 Translating Business Questions into Queries
Ejemplo de pregunta → SQL: *“¿Qué 5 productos tuvieron los mayores márgenes?”*


In [ ]:
sql("""
WITH sales(book_id, sale_date, revenue, cost, campaign_id) AS (
  VALUES (1,'2025-06-01',200,110,501),(2,'2025-06-03',180,95,502),(3,'2025-06-10',160,85,503),
         (2,'2025-06-14',210,120,502),(1,'2025-06-20',140,80,501)
)
SELECT b.title,
       ROUND(100 * SUM(revenue - cost)/NULLIF(SUM(revenue),0),2) AS margin_pct
FROM sales s
JOIN books b USING(book_id)
GROUP BY b.title
ORDER BY margin_pct DESC
LIMIT 5;
""")

#### Exercise 5.2
Convierte la pregunta *“Which 5 products had the highest margins?”* en una consulta usando `ORDER BY` y `LIMIT` (adapta el ejemplo).

In [ ]:
# tu código

### 5.3 AI 🤖 – Using LLMs to turn business questions into queries
- Describe el dataset y la pregunta al LLM para obtener un primer borrador de SQL.
- Realiza **spot checks** y validaciones antes de usar.
- Copia aquí el prompt que usarías y pega la consulta sugerida (si aplica).


## Cierre
- Guarda tus consultas clave en un script comentado.
- Exporta una tabla final y redacta una recomendación ejecutiva.
- **Tarea:** rehacer los ejercicios usando tus propias tablas reales si existen.

## Siguientes Pasos
- **Próxima sesión:** Practica con tus compañeros - revisión requisitos para el proyecto del sprint 3.
- **Participación continua:** asistir a Co-Learning y a Sprint Focus, y usar los canales de Discord para hacer preguntas.
- **Recordatorios:** la grabación y recursos utilizados, se comparten al finalizar la sesión; en caso de necesitar apoyo adicional, agenda un 1:1.

_Última actualización: 2025-10-14 16:25 UTC_